In [ ]:
from datetime import datetime
import pandas as pd

import Graph_algorithms.a_star as a_star
import Graph_algorithms.graph_loader as loader
import Graph_algorithms.dijkstra as dijkstra
import Utilities.formatter as formatter

pd.options.display.float_format = '{:.2f}'.format

In [ ]:
graph_time = loader.load_weighted_graph(criterion='time')
graph_changes = loader.load_weighted_graph(criterion='change')

stop_coords = a_star.load_stop_coords()

In [ ]:
for conn in graph_time["Muchobór Wielki"]:
    print("Departure:", conn["departure_time"], "Arrival:", conn["arrival_time"], "Line:", conn["line"])

In [ ]:
start_time = datetime.strptime("23:38:00", "%H:%M:%S")
total_cost, path, run_time = dijkstra.dijkstra_min_time(graph_time, "Kowale (Stacja kolejowa)", "Kochanowskiego", start_time)

df = formatter.format_schedule_df(path, criterion="time", start_time=start_time)  
print(df.to_string(index=False))

print("\n\n\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME"],
    "value": [total_cost, run_time]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

In [ ]:
start_time = datetime.strptime("23:08:00", "%H:%M:%S")
total_cost, path, run_time = a_star.a_star_min_time(graph_time, stop_coords, "Muchobór Wielki", "Mroźna", start_time, max_speed=15)

df = formatter.format_schedule_df(path, criterion="time", start_time=start_time)  
print(df.to_string(index=False))

print("\n\n\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME"],
    "value": [total_cost, run_time]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

In [ ]:
start_time = datetime.strptime("08:00:00", "%H:%M:%S")
total_cost, path, run_time = a_star.a_star_min_changes(graph_changes, "Muchobór Wielki", "Mroźna",  start_time)

df = formatter.format_schedule_df(path, criterion="change", start_time=start_time)  
print(df.to_string(index=False))

print("\n\n\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME"],
    "value": [total_cost, run_time]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))